<a href="https://colab.research.google.com/github/RamsesMDLC/Smolagent_Project_1/blob/main/Smolagents_Project_1_YT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
x = 3
y = 9
z = x + y
print(z)

12


In [8]:
!pip install smolagents[toolkit]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.2 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [10]:
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get('HF_TOKEN')  # Retrieves the token securely
if hf_token:
    login(hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Token not found. Please add HF_TOKEN secret.")


Successfully logged in to Hugging Face!


In [15]:
#transformersModel:
  #model abstraction or wrapper that uses Hugging Face's Transformers library to load, manage, and run SOTA Transformer-based models.
  #Through "transformersModel", the Transformer-based LLMs are accessible and usable in frameworks like smolagents for tasks involving chat,...
  #...code generation, search
  #It abstracts away the complexity of tokenization, inference, and managing model pipelines, allowing agent frameworks (like smolagents)...
  #...to use powerful language models with simple APIs.

from smolagents import CodeAgent, DuckDuckGoSearchTool, TransformersModel

In [16]:
model = TransformersModel(model_id="HuggingFaceTB/SmolLM-135M-Instruct")
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]